Dado que as informações de bairro e cidade dos usuários do sistema de feiras são manualmente preenchidas, podem haver erros nestas informações. Para podermos fazer uma análise com mais qualidade, vamos aqui capturar o bairro e a cidade de todos os CEPs válidos presente no arquivo de compras e armazenar em um arquivo csv.<p>
A recuperação das informações se dá por meio de chamada a API grauita disponibilizada no site http://viacep.com.br/.<p>
A última execuação deste notebook foi em 9 de fevereiro de 2022, às 15∶28∶46.

In [18]:
import requests
from requests.adapters import HTTPAdapter, Retry
import pandas as pd
from unidecode import unidecode
import re

# Leitura dos CEPs
Vamos ler os CEPs presentes no arquivo, armazená-los em memória e formatá-los da maneira que a API espera (somente com dígitos) e com 8 bytes, sendo informados zeros não significativos caso necessário.

In [72]:
# Leitura do arquivo
df = pd.read_csv('arquivos/Arquivo inicial _202202191053.csv')

# Padronização do formato do CEP
df.dropna(subset=['cep_pedido'],inplace=True)
df['cep_pedido'] = df['cep_pedido'].apply(lambda cep: re.sub('[^0-9]','',cep))
df['cep_pedido'] = df['cep_pedido'].apply(lambda cep: f"{int(cep):08d}")

# Colocação dos CEPs únicos em lista
ceps = df[df['cep_pedido'].notnull()]['cep_pedido'].unique()

# Busca do bairro e da cidade
Acessando a API para descobrir o bairro e a cidade de cada CEP. CEPs inválidos serão descartados. É possível saber que um CEP é inválido pelo retorno da API: não é retornado nome ou bairro para CEPs inválidos.

In [73]:
s = requests.Session()
retries = Retry(total=5, backoff_factor=1, status_forcelist=[ 502, 503, 504 ])
s.mount('http://', HTTPAdapter(max_retries=retries))

# Primeiramente vamos definir uma função que busca o bairro e a cidade
def busca_end(cep):
    url = 'https://viacep.com.br/ws/' + cep + '/json/'
    content = s.get(url)
    
    if content.status_code == 200:
        py_content = content.json()
        if ('bairro' in py_content.keys()) and ('localidade' in py_content.keys()):
            return py_content['bairro'],py_content['localidade']

    return None,None

Agora vamos acionar a função para cada CEP.

In [80]:
ends = list()
i = int()

for cep in ceps:
    bairro,cidade = busca_end(cep)
    if bairro and cidade:
        ends.append([cep,unidecode(bairro.lower()),unidecode(cidade.lower())])
    
    i += 1
    if i % 50 == 0:
        print(f"CEPs tratados: {i}")

CEPs tratados: 50
CEPs tratados: 100
CEPs tratados: 150
CEPs tratados: 200
CEPs tratados: 250
CEPs tratados: 300
CEPs tratados: 350
CEPs tratados: 400
CEPs tratados: 450
CEPs tratados: 500
CEPs tratados: 550
CEPs tratados: 600
CEPs tratados: 650
CEPs tratados: 700
CEPs tratados: 750


In [81]:
enderecos = pd.DataFrame(ends,columns=['cep','bairro','cidade'])
enderecos

,cep,bairro,cidade
0,60150060,aldeota,fortaleza
1,60320105,sao gerardo,fortaleza
2,60713480,manoel satiro,fortaleza
3,60431075,rodolfo teofilo,fortaleza
4,60430660,rodolfo teofilo,fortaleza
...,...,...,...
749,60040420,fatima,fortaleza
750,04119000,vila mariana,sao paulo
751,60543322,bom jardim,fortaleza
752,60416020,jardim america,fortaleza


# Salvando arquivo csv

In [83]:
enderecos.to_csv('arquivos/Endereços dos CEPs.csv',index=False)